In [ ]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode

secao2020 = pd.read_csv('../data/votacao_secao-municipio_2020_sudeste_sp_sao_jose_dos_campos.csv', low_memory=False, delimiter=';',encoding='latin-1')

# Use nunique() para contar o número de seções únicas
numero_de_secoes = secao2020['nr_secao'].nunique()

# Exiba o resultado
print(f"Existem {numero_de_secoes} seções eleitorais diferentes no arquivo.")

secoes_unicas = secao2020[['nr_zona','nr_secao', 'nm_local_votacao', 'ds_local_votacao_endereco']].drop_duplicates()

# Substitua 'SUA_CHAVE_DO_OPENCAGE' pela chave de API real que você obteve
sua_chave_do_opencage = '20728ac7e34a475ebe366dd4f122d06f'

# Inicialize o geocoder do OpenCage
geocoder = OpenCageGeocode(sua_chave_do_opencage)

# Função para obter as coordenadas (latitude e longitude) a partir do endereço
def obter_coordenadas_opencage(endereco):
    try:
        result = geocoder.geocode(endereco)
        if result and len(result):
            latitude = result[0]['geometry']['lat']
            longitude = result[0]['geometry']['lng']
            return latitude, longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erro ao geocodificar: {e}")
        return None, None

# Aplique a função para obter as coordenadas a partir dos endereços no DataFrame
secoes_unicas['latitude'], secoes_unicas['longitude'] = zip(*secoes_unicas['ds_local_votacao_endereco'].apply(obter_coordenadas_opencage))


# Exiba o DataFrame resultante
print(secoes_unicas)

# Especifique o caminho e o nome do arquivo CSV de saída
caminho_saida_csv = '../export/secoes_unicas.csv'

# Exporte o DataFrame para o arquivo CSV
secoes_unicas.to_csv(caminho_saida_csv, index=False)